## Introduction

In this project, we will analyze the spread of the new corona virus (nCov). We will use two datasets:
    - The John Hopkins University's dataset which contains aggregated daily data for confirmed cases, deaths and recovered patients. 
    https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
    - The DXY.cn's google sheet which contains information of about 1000 patients. 
    https://docs.google.com/spreadsheets/d/1jS24DjSPVWa4iuxuD4OAXrE3QeI8c9BC1hSlqr-NMiU/edit#gid=1187587451


Son Huynh
29.02.2020

About getting dataset:

The reddit's datasets group is a good place to look for datasets. For example, you can check out useful sources for nCov data from here: https://www.reddit.com/r/datasets/comments/exnzrd/coronavirus_datasets/

Reddit in general is great for getting tips and browsing discussions. Some of my favorite subreddits are dataisbeautiful, datascience, MachineLearning.


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
url_case = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
url_death = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
url_cured = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

Let's take a look at the case time series first

In [139]:
case = pd.read_csv(url_case, index_col=[0, 1, 2, 3])
cured = pd.read_csv(url_cured, index_col=[0, 1, 2, 3])
deaths = pd.read_csv(url_death, index_col=[0, 1, 2, 3])

case.head()


,,,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
Province/State,Country/Region,Lat,Long,,,,,,,,,,,,,,,,,,,,,
Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,...,989,989,989,990,990,990,990,990,990,990
Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,...,400,400,410,410,411,413,414,414,418,418
Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,132,147,182,211,...,576,576,576,576,576,576,576,576,576,576
Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,80,84,101,120,...,294,294,296,296,296,296,296,296,296,296
Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,19,24,26,29,...,91,91,91,91,91,91,91,91,91,102


Since the John Hopkins dataset contains only aggregated data, we need to transform it into a format that allows us to answer more questions. You can see the ideal dataframe structure in the picture below.

You will need to:
- Stack the dataframe so that each row represents one date in a location. 
- Remove the rows where the cumulative number of cases is zero.
- Make a new column `location` to combine `prov_state` and `country`.
- Make a new column `new_case` to derive the new case number from cumulative case number.
- Convert the date column to datetime object.

After transforming data for number of cases, you can do the same for number of deaths and cureds (or write a function to transform 3 datasets). Then you can use `pd.merge()` to merge them into one dataframe. Hint: try merging `case` with `death` first and then merge that with `cured`. You should use a `left` merge on `['location', 'prov_state', 'country', 'lat', 'long', 'date']` columns

![title](desired_format.png)

In [182]:
# stack to represent a location per row
stack = case.stack()
stack = stack.reset_index()

# rename columns
stack.columns = ['ProvinceState','CountryRegion', 'Lat', 'Long','date','cum_cases']
stack = stack[stack.cum_cases>0]

# create a location column combining ProviceState or CountryRegion
stack['location'] = np.where(stack.ProvinceState.notna(), stack.ProvinceState, stack.CountryRegion)

# create a new cases column
stack['new_cases'] = np.where(stack.cum_cases.diff(periods=1)>=0, stack.cum_cases.diff(periods=1), stack.cum_cases)

# convert the date column to datetime object
stack.date = pd.to_datetime(stack.date, format='%m/%d/%y', errors='coerce')

# rearranging the columns and assigning to dataframe variable.
df = stack[['location', 'ProvinceState', 'CountryRegion', 'Lat', 'Long', 'date', 'cum_cases', 'new_cases']]

df.iloc[500:510]

,location,ProvinceState,CountryRegion,Lat,Long,date,cum_cases,new_cases
502,Henan,Henan,Mainland China,33.88202,113.614,2020-02-09,1033,52.0
503,Henan,Henan,Mainland China,33.88202,113.614,2020-02-10,1073,40.0
504,Henan,Henan,Mainland China,33.88202,113.614,2020-02-11,1105,32.0
505,Henan,Henan,Mainland China,33.88202,113.614,2020-02-12,1135,30.0
506,Henan,Henan,Mainland China,33.88202,113.614,2020-02-13,1169,34.0
507,Henan,Henan,Mainland China,33.88202,113.614,2020-02-14,1184,15.0
508,Henan,Henan,Mainland China,33.88202,113.614,2020-02-15,1212,28.0
509,Henan,Henan,Mainland China,33.88202,113.614,2020-02-16,1231,19.0
510,Henan,Henan,Mainland China,33.88202,113.614,2020-02-17,1246,15.0
511,Henan,Henan,Mainland China,33.88202,113.614,2020-02-18,1257,11.0


In [183]:
# data for cured and deaths are arranged in the same manner as cases
cured.head()

,,,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
Province/State,Country/Region,Lat,Long,,,,,,,,,,,,,,,,,,,,,
Anhui,Mainland China,31.8257,117.2264,0,0,0,0,0,0,0,2,2,3,...,712,744,792,821,868,873,917,936,956,970
Beijing,Mainland China,40.1824,116.4142,0,0,1,2,2,2,4,4,4,5,...,215,235,248,257,271,276,282,288,297,297
Chongqing,Mainland China,30.0572,107.8740,0,0,0,0,0,0,0,1,1,1,...,372,384,401,422,438,450,469,490,502,512
Fujian,Mainland China,26.0789,117.9874,0,0,0,0,0,0,0,0,0,0,...,199,218,228,235,243,247,255,260,270,277
Gansu,Mainland China,36.0611,103.8343,0,0,0,0,0,0,0,0,0,0,...,80,81,81,82,82,84,85,86,87,87


In [185]:
stack = deaths.stack()
stack = stack.reset_index()

# rename columns
stack.columns = ['ProvinceState','CountryRegion', 'Lat', 'Long','date','cum_deaths']
# stack = stack[stack.cum_deaths>0]

# create a location column combining ProviceState or CountryRegion
stack['location'] = np.where(stack.ProvinceState.notna(), stack.ProvinceState, stack.CountryRegion)

# create a new deaths column
stack['new_deaths'] = np.where(stack.cum_deaths.diff(periods=1)>=0, stack.cum_deaths.diff(periods=1), stack.cum_deaths)

# convert the date column to datetime object
stack.date = pd.to_datetime(stack.date, format='%m/%d/%y', errors='coerce')

# rearrange dataframe.
stack = stack[['location', 'ProvinceState', 'CountryRegion', 'Lat', 'Long', 'date', 'cum_deaths', 'new_deaths']]

# merge deaths with cases
df = df.merge(stack)
df.head(20)

,location,ProvinceState,CountryRegion,Lat,Long,date,cum_cases,new_cases,cum_deaths,new_deaths
0,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,1.0,0,0.0
1,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-23,9,8.0,0,0.0
2,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-24,15,6.0,0,0.0
3,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-25,39,24.0,0,0.0
4,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-26,60,21.0,0,0.0
5,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-27,70,10.0,0,0.0
6,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-28,106,36.0,0,0.0
7,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-29,152,46.0,0,0.0
8,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-30,200,48.0,0,0.0
9,Anhui,Anhui,Mainland China,31.8257,117.2264,2020-01-31,237,37.0,0,0.0


In [188]:
stack = cured.stack()
stack = stack.reset_index()

# rename columns
stack.columns = ['ProvinceState','CountryRegion', 'Lat', 'Long','date','cum_cured']

# create a location column combining ProviceState or CountryRegion
stack['location'] = np.where(stack.ProvinceState.notna(), stack.ProvinceState, stack.CountryRegion)

# create a new cases column
stack['new_cured'] = np.where(stack.cum_cured.diff(periods=1)>=0, stack.cum_cured.diff(periods=1), stack.cum_cured)

# convert the date column to datetime object
stack.date = pd.to_datetime(stack.date, format='%m/%d/%y', errors='coerce')

# rearrange dataframe.
stack = stack[['location', 'ProvinceState', 'CountryRegion', 'Lat', 'Long', 'date', 'cum_cured', 'new_cured']]

# merge cured with cases
df = df.merge(stack)

In [198]:
df.iloc[3150:3200]


df.groupby('location').new_cases.sum().sort_values(ascending=False).head(50)

location
Hubei                           67466.0
South Korea                      6088.0
Italy                            3857.0
Iran                             3513.0
Guangdong                        1351.0
Henan                            1272.0
Zhejiang                         1215.0
Hunan                            1018.0
Anhui                             990.0
Jiangxi                           935.0
Shandong                          758.0
Diamond Princess cruise ship      705.0
Jiangsu                           631.0
Chongqing                         576.0
Sichuan                           539.0
Heilongjiang                      481.0
Germany                           481.0
Beijing                           418.0
Japan                             407.0
France                            377.0
Shanghai                          339.0
Hebei                             318.0
Fujian                            296.0
Spain                             259.0
Guangxi                        